## Project 3: Trading Bot

### Prompt

*"You've just determined a winning stock market trading strategy. The problem? It requires you to make trades extremely precisely -- with little room for error. Build your own algorithmic trading bot -- capable of responding to incoming market data in real-time."*

### Approach

I'm going to recreate [this notebook](https://github.com/ORE93/Automated-Algorithmic-Trading-Bot/blob/main/Algo_trade.ipynb) and then see if I can improve on their results. Their solution relies on the [Alpaca trade API](https://docs.alpaca.markets/docs/getting-started-with-trading-api), which in turn relies on Python 3.10, so I switched to a new virtual environment using [Miniconda](https://docs.conda.io/en/latest/miniconda.html) and [these packages](./requirements-conda.txt).

In [8]:
import pandas
import numpy
import json
from alpaca.trading.client import TradingClient

In [12]:
with open('./api/alpaca-paper.json') as keyfile: api = json.load(keyfile)
endpoint = TradingClient(api['KEY'], api['SECRET'], paper=True)

In [11]:
# client.get_account()